## Working_with_WRF_CHEM_part3-compile_wrf_exe

- for running wrf.exe after wps, it is found that no link for wrf.exe and real.exe was made during the compilation of wrf-chem.
- so now need of fresh compilation of wrf alone to make it work
- the compilation involves only setting environment, installing dependent packages and running, ```
./configure``` then ```./compile em_real``` and ```./compile emi_conv```
- the first step of .configure ended with error saying can't find netcdf in the path, the path specified is wrong one. So the file ```sudo nano bash.bashrc``` was edited removed the extract two line for NETCDF with set for different installation location. These lines were uncommented with maintaining netcdf path for /usr/local. this was came to know from ```nc-config --all``` to know the netcdf installed location. But after editing also the configure returned same error.
- after a reboot the configure was repeated and it ended without any error, so it can be concluded that the setting environment edit in bash.bashrc will take effect after a reboot.
- Then done ```./compile em_real```, but this time also compilation run for almost 20 minutes, but no real.exe is made in main folder. But its links are made elsewhere such as run and test folders. the common show is of this nature 
```bash
..../wrf_utils/./read_wrf_nc.f:790: undefined reference to `nf_open_'
..../wrf_utils/./read_wrf_nc.f:793: undefined reference to `nf_open_'
..../wrf_utils/./read_wrf_nc.f:802: undefined reference to `nf_inq_varid_'
..../wrf_utils/./read_wrf_nc.f:811: undefined reference to `ncvid_'
```
- so based on experiance in compiling PREP_CHEM_SRC, the netcdf path was edited with additional ```-lnetcdff```  and rerun compilation. 
- this time selecting serial, 23, compilation and editing configure.wrf, completed the compilation for 15 minutes and made wrf.exe and executables and there is no error as seen in earlier compilation.
- then executed the commend for wrf chem, ```./compile emi_conv``` it also compiled with out any error and it made executables for emiss_conv.exe, this was not made in eaarlier case. 
- so now testing for parallel compiling with editing in configure.wrf for libnetdf and then made reptiation, but with dm+sm, it existed without genrating any exe files. 
- For genrating log file along with compile made a serch and ended up ```[here](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/Compile/wps_compile3.htm) and found a [treasure trove](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/compilation_tutorial.php#STEP4)``` for WRF compiling. This page comphrehensively explains numerous stpes of wrf comipling with library installation and most importatnly test the system requierment for wrf compiling.
- Using this guide made a attmept for testing the librariesa and env in the current system for wrf compiling. Made a test of 7+2 and all the test were exited without any error, in netcdf sum warning came but it also exited with success. Found mpich2 was older version than specified in the guide, current version is 1.4.1 and recommended in the guide is 3.0.1!.
- So based on this success of all tests, made configure and editing of configure.wrf and compiled wrf with ```./compile em_real >& compile.log``` and selected dmpar for runnning. 
- but this also exited with error, the error were ```/usr/local/include/mpi.h:1501:31: error: expected declaration specifiers or ‘...’ before ‘MPI_Comm’```, so concluded that new version of mpich2 has to be installed.
- Mpich2 were installed during making beowlf cluster creation processes as per [this](http://byobu.info/article/Building_a_simple_Beowulf_cluster_with_Ubuntu/) where, mpich2 was installed using ```sudo apt-get install mpich2```, and the verision was 1.4.1, needed for wrf compilation is 3.0.1 so it wass unstallaed by ```sudo apt-get remove mpich2```
- so followed the tutorial and compiled mpich using the commands 
```bash
tar xzvf mpich-3.0.4.tar.gz     #or just .tar if no .gz present
cd mpich-3.0.4
./configure --prefix=$DIR/mpich
make
sudo checkinstall
```
- to view the location of the intsalled packaeg run ```dpkg -L mpich```, it shows it in / folder, to consider the difference in c and bash shell a line was added into the file ```/etc/bash.bashrc``` as ```export mpich=/mpich/bin```.
- restart the computer and recompiling, but it was also a fail.
- found a installation manual from mpich compiled folder, it says `prefix=$DIR` makes compilation into referred folder, instead path ```/usr/local/bin``` is its default compiling location for mpich, that means giving no prefix makes problem to solve.
- The manual also shows instruction for setenv for Bash terminals it has to do it in this way ```PATH=/usr/local/bin:$PATH``` and ```export path```. now on ward it seems mpich is working properly, such as giveing ```which mpicc``` respond fine.
- But made confused attempt of recompileing and all... with a pretext that the mpich is not working properly, mainly due to the testing said in wrf compile tutorial is not working, the final step in mpirun made into a stand still and gives error after a long time such as ```MPICH2, mpirun > ssh: Could not resolve hostname```. But made a google search with this error and ofund [this](http://forums.opensuse.org/showthread.php/475510-MPICH2-mpirun-gt-ssh-Could-not-resolve-hostname) web page sauing mpirun has to run based on the editing in the file ```cat /etc/hosts```, it was actually edited durign bewolf cluster creation, so made the test with this mpi run command ```mpirun -np 1 -host localhost ./a.out``` this resolved problem.
- with out considering this problem wrf was compiled with 25 (dmpar) option and it compiled correctly with creation of eexcettables. running of mpirun with this code ```mpirun -np 1 -host localhost ./a.out``` will makes wrf to run in parellelll mode. hooorahe, this was wrf compilation with parrellel mode makes a happy ending!